# General

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import pandas as pd
from scipy.constants import c
import os
print('c =', c) # speed of light in m/s
data_folder = r'c:\Users\yaniv\Yehonathan TAU\PhyChemLab\interferometry_data'
os.chdir(data_folder)
print('Current working directory:', os.getcwd())

# Functions

In [ ]:
def load_data(file_path):
    data = pd.read_csv(file_path, sep='\t', names = ['wavelength nm', 'intensity'])
    data['frequency THz'] = c / data['wavelength nm'] * 1e-3 # convert to THz
    return data

def plot_comb(data, peaks = None, type = 'frec', title = None):
    if type == 'frec':
        plt.plot(data['frequency THz'], data['intensity'])
        if peaks is not None:
            plt.plot(data['frequency THz'][peaks], data['intensity'][peaks], 'x')
        plt.xlabel('Frequency [THz]')
        plt.ylabel('Intensity')
        if title is not None:
            plt.title(title)
        else:
            plt.title('Comb Spectrum | Frequency', fontsize=16)
    elif type == 'wavel':
        plt.plot(data['wavelength nm'], data['intensity'])
        if peaks is not None:
            plt.plot(data['wavelength nm'][peaks], data['intensity'][peaks], 'x')
        plt.xlabel('Wavelength [nm]')
        plt.ylabel('Intensity')
        if title is not None:
            plt.title(title)
        else:
            plt.title('Comb Spectrum | Wavelength')
    elif type == 'both':
        fig, axs = plt.subplots(2, 1, figsize=(10, 8))
        axs[0].plot(data['frequency THz'], data['intensity'])
        if peaks is not None:
            axs[0].plot(data['frequency THz'][peaks], data['intensity'][peaks], 'x')
        axs[0].set_xlabel('Frequency (THz)')
        axs[0].set_ylabel('Intensity')
        axs[0].set_title('Comb Spectrum | Frequency', fontsize=16)

        axs[1].plot(data['wavelength nm'], data['intensity'])
        if peaks is not None:
            axs[1].plot(data['wavelength nm'][peaks], data['intensity'][peaks], 'x')
        axs[1].set_xlabel('Wavelength (nm)')
        axs[1].set_ylabel('Intensity')
        axs[1].set_title('Comb Spectrum | Wavelength', fontsize=16)
        if title is not None:
            fig.suptitle(title, fontsize=20)
        else:
            fig.suptitle('Comb Spectrum', fontsize=20)

        plt.tight_layout()
    else:
        print('Invalid plot type. Use "frec" or "wavel"')
    plt.show()

def find_peaks_comb(data, threshold = 0.01,width = None, distance = 1):
    peaks, _ = find_peaks(data['intensity'], height = threshold, distance = distance, width = width)
    return peaks

def culc_frec_spacing(data, peaks, plot = False, toprint = False):
    frec_spacing = - np.diff(data['frequency THz'][peaks])
    frec_spacing_mean = np.mean(frec_spacing) # THz
    frec_spacing_std = np.std(frec_spacing) # THz

    delta_x = c / frec_spacing_mean * 1e-6 # convert to um
    delta_x_err = c / frec_spacing_std * 1e-6 # convert to um
    
    if toprint:
        print('Mean frequency spacing: {:.3f} ± {:.3f} THz'.format(frec_spacing_mean, frec_spacing_std))
        print('Δx = {:.3f} ± {:.3f} μm\n'.format(delta_x, delta_x_err))

    if plot:
        plt.hist(frec_spacing, bins = 20)
        plt.xlabel('Frequency Spacing [THz]')
        plt.ylabel('Counts')
        plt.title('Frequency Spacing Histogram')
        plt.show()

    return delta_x, delta_x_err

def culc_file(file_path, plot1 = False, plot2 = False, threshold = 0.01,width = None, distance = 1):
    data = load_data(file_path)
    peaks = find_peaks_comb(data, threshold = threshold, width = width, distance = distance)
    if plot1:
        if plot1 == 'frec':
            text_type = 'Frequency'
        elif plot1 == 'wavel':
            text_type = 'Wavelength'
        else:
            text_type = 'Both'
        title = 'Comb Spectrum | ' + text_type + ' | ' + os.path.splitext(os.path.basename(file_path))[0]
        plot_comb(data, peaks, type = plot1, title = title)
    delta_x, delta_x_err = culc_frec_spacing(data, peaks, plot2, toprint= bool(plot1))
    return delta_x, delta_x_err

def extract_reflective_index(mesure_file, reference_file, d, plot = False): # d in um!!!

    delta_x_ref, delta_x_err_ref = culc_file(reference_file, plot1 = plot) # um
    delta_x_mes, delta_x_err_mes = culc_file(mesure_file, plot1 = plot) # um

    l = delta_x_mes - delta_x_ref # um
    l_err = np.sqrt(delta_x_err_ref**2 + delta_x_err_mes**2) # um

    delta_n = l / (2* d ) # refractive index difference
    delta_n_err = l_err / (2 * d) # refractive index difference error

    n = 1 + delta_n # refractive index
    n_err = delta_n_err # refractive index error
    
    if plot:
        print('l = {:.3f} ± {:.3f} μm'.format(l, l_err))
        print('Δn = {:.3f} ± {:.3f}'.format(delta_n, delta_n_err))
        print('n = {:.3f} ± {:.3f}'.format(n, n_err))

    return n, n_err

# Week 2

## 2.6 Quantify the Optical Path Difference of the Arms

In [ ]:
file1 = r"434DH.tab"
data1 = load_data(file1)
peaks = find_peaks_comb(data1)
plot_comb(data1, peaks, type = 'frec')
delta_x1, delta_x_err1 = culc_frec_spacing(data1, peaks, plot = False, toprint = True)

## 2.7 Quantify the Interferometer Translation

In [ ]:
file2 = '434DH.tab'
delta_x2, delta_x_err2 = culc_file(file2, plot1 = 'frec')
file3 = '434DH.tab'
delta_x3, delta_x_err3 = culc_file(file3, plot1 = 'frec')

l2 = delta_x2 - delta_x1
l3 = delta_x3 - delta_x1
l2_err = np.sqrt(delta_x_err1**2 + delta_x_err2**2)
l3_err = np.sqrt(delta_x_err1**2 + delta_x_err3**2)

print('l2 = {:.3f} ± {:.3f} μm'.format(l2, l2_err))
print('l3 = {:.3f} ± {:.3f} μm'.format(l3, l3_err))

## 2.8 Intensity Modulation of the Arms

## 2.9 Demonstrate the Law of Energy Conservation


In [ ]:
file_energy1 = '434DH.tab'
file_energy2 = '434DH.tab'
data_energy1 = load_data(file_energy1)
data_energy2 = load_data(file_energy2)

plt.plot(data_energy1['wavelength nm'], data_energy1['intensity'], label = 'interference pattern #1')
plt.plot(data_energy2['wavelength nm'], data_energy2['intensity'], label = 'interference pattern #2')
plt.xlabel('Wavelength [nm]')
plt.ylabel('Intensity')
plt.title('Interference Patterns - Energy conservation')
plt.legend()
plt.show()

## 2.10 Measure the Index of Refraction of a Solid Sample

In [ ]:
reference_file = '434DH.tab'
mesure_file = '434DH.tab'
d = 180 # sample thickness in um 
extract_reflective_index(mesure_file, reference_file, d, plot = 'frec')

## 2.11 Measure the Index of Refraction Dependence on the Concentration of a Solution

In [ ]:
concentration_file = 'file_name.xlsx'
concentration_data = pd.read_excel(concentration_file)
data_folder = 'data_folder_path'
reference_file = 'reference_file_name.tab'
d = 1e6 # cuvettes thickness in um

n_list = []
n_err_list = []

for file in concentration_data['file_name']:
    file_path = os.path.join(data_folder, file)
    n, n_err = extract_reflective_index(file_path, reference_file, d, plot = False)
    n_list.append(n)
    n_err_list.append(n_err)

concentration_data['n'] = n_list
concentration_data['n_err'] = n_err_list

# plt.plot(concentration_data['concentration'], concentration_data['n'], '.')
plt.errorbar(concentration_data['concentration'], concentration_data['n'], yerr=concentration_data['n_err'], fmt='.', capsize=5)
plt.xlabel('Concentration [M]')
plt.ylabel('Refractive Index')
plt.title('Refractive Index vs. Concentration')
plt.show()
